In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:12:34 of 0:12:35 remaining]
Chain 1:  10% [0:00:58 of 0:01:05 remaining]
Chain 1:  20% [0:00:55 of 0:01:09 remaining]
Chain 1:  30% [0:00:51 of 0:01:12 remaining]
Chain 1:  40% [0:00:44 of 0:01:13 remaining]
Chain 1:  50% [0:00:36 of 0:01:12 remaining]
Chain 1:  60% [0:00:29 of 0:01:13 remaining]
Chain 1:  70% [0:00:22 of 0:01:14 remaining]
Chain 1:  80% [0:00:15 of 0:01:15 remaining]
Chain 1:  90% [0:00:08 of 0:01:15 remaining]
Chain 1: 100% [0:00:00 of 0:01:16 remaining]

Chain 2:   0% [0:00:02 of 0:00:02 remaining]
Chain 2:  10% [0:00:55 of 0:01:01 remaining]
Chain 2:  20% [0:00:52 of 0:01:06 remaining]
Chain 2:  30% [0:00:47 of 0:01:07 remaining]
Chain 2:  40% [0:00:44 of 0:01:14 remaining]
Chain 2:  50% [0:00:39 of 0:01:18 remaining]
Chain 2:  60% [0:00:31 of 0:01:19 remaining]
Chain 2:  70% [0:00:24 of 0:01:19 remaining]
Chain 2:  80% [0:00:16 of 0:01:19 remaining]
Chain 2:  90% [0:00:08 of 0:01:22 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[25.22 114.084 0.896618; 25.7528 113.547 0.916072; … ; 28.5014 113.65 0.911957; 24.1392 113.65 0.911957]

[27.831 113.482 0.919646; 23.6281 113.482 0.919646; … ; 31.4312 111.99 0.943154; 31.6522 111.99 0.943154]

[26.6984 115.87 0.865789; 24.1344 115.219 0.881692; … ; 26.9715 116.124 0.851844; 25.8078 116.124 0.851844]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean        SD       Naive SE       MCSE       ESS  
     s2  26.1672521 2.00886054 0.0122255359 0.020344292 9000.000
beta[1] 113.9595610 1.90357562 0.0115847923 0.036677465 2693.653
beta[2]   0.9033249 0.04193686 0.0002552196 0.000802284 2732.345

Quantiles:
            2.5%         25.0%        50.0%        75.0%       97.5%   
     s2  22.52116028  24.77478698  26.07721770  27.45327143  30.3712559
beta[1] 110.16783220 112.70105922 113.99169460 115.23797033 117.7366705
beta[2]   0.82052889   0.87549115   0.90261393   0.93096772   0.9871789



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*